In [1]:
import pandas as pd

In [2]:
# df_action = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv', parse_dates=['sess_dt'])
df_transaction = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv', parse_dates=['de_dt'])
# df_client_demo = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv')
df_product = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv')

#### CF - users based on the prefrences of other users
- First step: To find similar users or items
- Second step : To predict the products that are not yet purchased by a user.

##### Question
1. How do you determine which users or items are similar to one another? (어떤 사용자 또는 물건이 서로 유사한지 어떻게 결정하나요?)
2. Given that you known which users are similar, how do you determine the rating that a user would give to an item based on the ratings of similar users? (어떤 사용자가 유사한지 알고 있는 경우, 사용자가 유사한 사용자의 등급을 기준으로 항목에 부여할 등급을 어떻게 결정하나요?)
3. How do you measure the accuracy of the ratings you calculate?(계산한 등급의 정확도를 어떻게 측정하나요?)
    - RMSE, metric to measure the accuracy; MAE

In [3]:
df_transaction.tail(5)

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
599956,35311,2839,3,B03,0339,2019-09-27,14:09,3500,1
599957,35311,2839,4,B03,0339,2019-09-27,14:09,3600,2
599958,35311,2839,5,B03,0339,2019-09-27,14:09,3500,1
599959,35311,2839,2,B03,0339,2019-09-27,14:09,4500,1
599960,35311,2839,6,B03,0324,2019-09-27,14:09,6800,1


In [4]:
# Using groupby to group the product and getting the count by clint
count_by_pd = pd.DataFrame(df_transaction.groupby(['pd_c'])['clnt_id'].count())

# Arranging the output in descending order and taking head to get the top 5 most popular
count_by_pd.sort_values('clnt_id', ascending=False).head(5)

,clnt_id
pd_c,
unknown,17652
1213,15518
0347,12914
0964,11115
1395,11083


In [5]:
df_product.loc[df_product['pd_c'] == 1213]

,pd_c,clac_nm1,clac_nm2,clac_nm3
1212,1213,Snack Foods,Snacks,General Snacks


In [6]:
df_product.loc[df_product['pd_c'] == 347]

,pd_c,clac_nm1,clac_nm2,clac_nm3
346,347,Dairy Products,Milk,Fresh Milk


In [7]:
df_product.loc[df_product['pd_c'] == 964]

,pd_c,clac_nm1,clac_nm2,clac_nm3
963,964,Meats,Eggs,Chicken Eggs


In [8]:
df_product.loc[df_product['pd_c'] == 1395]

,pd_c,clac_nm1,clac_nm2,clac_nm3
1394,1395,Substitute Foods,Instant Noodles,Ramens


In [9]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599961 entries, 0 to 599960
Data columns (total 9 columns):
clnt_id      599961 non-null int64
trans_id     599961 non-null int64
trans_seq    599961 non-null int64
biz_unit     599961 non-null object
pd_c         599961 non-null object
de_dt        599961 non-null datetime64[ns]
de_tm        599961 non-null object
buy_am       599961 non-null int64
buy_ct       599961 non-null int64
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 41.2+ MB


In [10]:
df_transaction.loc[df_transaction['pd_c'] == 'unknown', 'pd_c'] = '0000' 

In [11]:
df_transaction['pd_c'] = df_transaction['pd_c'].astype(int)

In [13]:
df = df_transaction[['clnt_id', 'pd_c', 'buy_ct']]
df.head()

,clnt_id,pd_c,buy_ct
0,21922,0,1
1,21279,0,1
2,39423,0,1
3,18362,0,1
4,39423,565,1


In [14]:
# group by and count
counts_df = pd.DataFrame(df.groupby('pd_c')['buy_ct'].count())
counts_ascending_df = counts_df.sort_values('buy_ct', ascending=False)
counts_ascending_df.head()

,buy_ct
pd_c,
0,17652
1213,15518
347,12914
964,11115
1395,11083


In [15]:
# filter and return most valued product
most_valued = counts_ascending_df.head(5).index.tolist()
most_valued

[0, 1213, 347, 964, 1395]

In [16]:
df_product.loc[most_valued]

,pd_c,clac_nm1,clac_nm2,clac_nm3
0,1,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1213,1214,Snack Foods,Snacks,Popcorns
347,348,Dairy Products,Milk,Functional Milk
964,965,Meats,Eggs,Processed Chicken Eggs
1395,1396,Substitute Foods,Retort Pouches,Instant Bowl Of Rice With Toppings


In [ ]:
def based_on_count(df, product_df, n):
    """
    Based on count
    """
    # group by and count
    counts_df = pd.DataFrame(df.groupby('pd_c')['buy_ct'].count())
    counts_ascending_df = counts_df.sort_values('buy_ct', ascending=False)
    
    # filter and return most valued product
    most_valued = counts_ascending_df.head(n).index.tolist()
    
    return df_product.loc[most_valued]

In [ ]:
def main():
    # Inputs. I want a feedback of 5 products.
    n = 5
    df = df_transaction[['clnt_id', 'pd_c', 'buy_ct']]
    product_df = df_product

    # calculating
    print('Based on counts')
    print(based_on_count(df, product_df, n))

if __name__ == "__main__":
    main()